In [ ]:
!pip install selenium

In [ ]:
#Scrape Reviews From Amazon

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb
import numpy as np

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

In [ ]:
driver.get('https://www.amazon.in/Apple-iPhone-15-128-GB/product-reviews/B0CHX1W1XY/ref=cm_cr_getr_d_paging_btm_next_10?ie=UTF8&reviewerType=all_reviews')

In [ ]:
html_data = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
i = 0
names = []
ratings = []
rating_dates = []
titles = []
reviews_text = []
url = 'https://www.amazon.in/Apple-iPhone-15-128-GB/product-reviews/B0CHX1W1XY/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber='
while url != None:
    i = i + 1
    url = 'https://www.amazon.in/Apple-iPhone-15-128-GB/product-reviews/B0CHX1W1XY/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber='+str(i)
    driver.get(url)
    time.sleep(10)
    html_data = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = html_data.find_all('div',{'data-hook':'review'})
    for review in reviews:
        name = review.find('span',{'class':'a-profile-name'}).text
        names.append(name.strip())
        rating = review.find('span',{'class':'a-icon-alt'}).text
        ratings.append(rating)
        rating_date = review.find('span',{'data-hook':'review-date'}).text
        rating_dates.append(rating_date)
        title = review.find('a',{'data-hook':'review-title'}).text
        titles.append(title)
        review_text = review.find('span',{'data-hook':'review-body'}).text
        reviews_text.append(review_text)
    url_check = html_data.find('li',{'class':'a-last'})
    if url_check == None:

        url = None

    else:
        url = 'https://www.amazon.in' + html_data.find('li',{'class':'a-last'}).a['href']
    print(url)


https://www.amazon.in/Apple-iPhone-15-128-GB/product-reviews/B0CHX1W1XY/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber=2&reviewerType=all_reviews
https://www.amazon.in/Apple-iPhone-15-128-GB/product-reviews/B0CHX1W1XY/ref=cm_cr_arp_d_paging_btm_3?ie=UTF8&pageNumber=3&reviewerType=all_reviews


KeyboardInterrupt: 

In [ ]:
# Create a DataFrame from the collected data
data = pd.DataFrame({
    'name': names,
    'rating': ratings,
    'rating_date': rating_dates,
    'title': titles,
    'review_text': reviews_text
})
data.rating.value_counts()

rating
5.0 out of 5 stars    10
Name: count, dtype: int64

In [ ]:
data.to_csv('5_StarReviews_Mouse.csv')

# Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
#, '4_StarReviews_Mouse.csv', '3_StarReviews_Mouse.csv', '2_StarReviews_Mouse.csv', '1_StarReviews_Mouse.csv'
# List of CSV files
dataframes_list = ['5_StarReviews_Mouse.csv']

# Initialize an empty list to hold DataFrames
frames = []

# Iterate over the list of file paths
for storepath in dataframes_list:
    # Read each CSV file into a DataFrame
    frame = pd.read_csv(f"/content/{storepath}", index_col=0)

    # Drop specified columns
    frame = frame.drop(columns=['name', 'rating_date', 'title'])

    # Append the DataFrame to the list
    frames.append(frame)

# Concatenate all DataFrames in the list
merged_df = pd.concat(frames, ignore_index=True)

In [ ]:
#for single File
merged_df = pd.read_csv('/content/5_StarReviews_Mouse.csv', index_col=0)
merged_df.head()


,name,rating,rating_date,title,review_text
0,Rahul,5.0 out of 5 stars,Reviewed in India on 26 June 2024,5.0 out of 5 stars\nTerrific product for the p...,\nAbsolutely amazed by the quality of the prod...
1,Jatin,5.0 out of 5 stars,Reviewed in India on 19 July 2024,5.0 out of 5 stars\nSoft clicks\n,"\nGood quality mouse, with decent response tim..."
2,Ajay,5.0 out of 5 stars,Reviewed in India on 19 July 2024,5.0 out of 5 stars\nGreat Product\n,\nThis mouse comes up with great bulid quality...
3,shakila,5.0 out of 5 stars,Reviewed in India on 25 July 2024,5.0 out of 5 stars\nReliable and High-Precisio...,\nThe ZEBRONICS Zeb-Jaguar Wireless Mouse is f...
4,Tanvi Maheshwari,5.0 out of 5 stars,Reviewed in India on 25 July 2024,5.0 out of 5 stars\nConnectivity seamless\n,"\nEnjoy seamless connectivity with 2.4GHz, Plu..."


In [ ]:
merged_df = merged_df.drop(columns=['name', 'rating_date', 'title'])


In [ ]:
merged_df.head()

,rating,review_text
0,5.0 out of 5 stars,\nAbsolutely amazed by the quality of the prod...
1,5.0 out of 5 stars,"\nGood quality mouse, with decent response tim..."
2,5.0 out of 5 stars,\nThis mouse comes up with great bulid quality...
3,5.0 out of 5 stars,\nThe ZEBRONICS Zeb-Jaguar Wireless Mouse is f...
4,5.0 out of 5 stars,"\nEnjoy seamless connectivity with 2.4GHz, Plu..."


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb



# Convert 'rating' column to numeric, extract numeric values, and convert to integer
merged_df['rating'] = merged_df['rating'].astype(str).str.extract(r'(\d+\.\d+)')
merged_df['rating'] = pd.to_numeric(merged_df['rating'], errors='coerce').fillna(0).astype(int)

# Define function to get sentiment label
def get_sentiment_label(rating):
    if pd.isna(rating):
        return 'unknown'
    elif rating <= 3:
        return 'negative'

    else:
        return 'positive'

# Apply the function to the 'rating' column
merged_df['rating'] = merged_df['rating'].apply(get_sentiment_label)

# Handle NaN values in 'review_text'
merged_df['review_text'] = merged_df['review_text'].fillna('No Review')

# Save to CSV
merged_df.to_csv('Merged_reviews.csv', index=False)

# Prepare data for model training
data = merged_df
X = data['review_text']
y = data['rating']

# Vectorization
vectorizer = CountVectorizer(stop_words='english')
X_vec = vectorizer.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_vec, y_encoded, test_size=0.3, random_state=42)

# Train XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report for more details
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 1.00
              precision    recall  f1-score   support

    positive       1.00      1.00      1.00        30

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:10:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
# Function for predicting new reviews
def predict_sentiment(review):
    review_vec = vectorizer.transform([review])
    prediction_encoded = model.predict(review_vec)
    prediction = label_encoder.inverse_transform(prediction_encoded)
    return prediction[0]

# Example usage of prediction function
new_review = input("Enter a review to predict sentiment: ")
predicted_sentiment = predict_sentiment(new_review)
print(f"The predicted sentiment is: {predicted_sentiment}")


Enter a review to predict sentiment: its good
The predicted sentiment is: positive


In [ ]:
!pip install transformers
from transformers import pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

# **Review Summerization**

In [ ]:
# Load reviews from the CSV file
file_path = "/content/5_StarReviews_Mouse.csv"
df = pd.read_csv(file_path)

# Combine all reviews into a single text
all_reviews = ' '.join(df['review_text'].fillna(''))

# Generate the summary
summary = summarizer(all_reviews, max_length=500, min_length=30, do_sample=False)

# Print the summary
print("Summary:")
print(summary[0]['summary_text'])


Summary:
 The ZEBRONICS Zeb-Jaguar Wireless Mouse is fantastic! It offers smooth and precise tracking, and the USB Nano Receiver makes it super convenient to use . The mouse is lightweight and ergonomic in your hands which is one of the most important thing and it does the job properly in that department .


In [ ]:
# Load reviews from the CSV file
file_path = "/content/5_StarReviews_Mouse.csv"
df = pd.read_csv(file_path)

# Combine all reviews into a single text
all_reviews = ' '.join(df['review_text'].fillna(''))

# Generate the summary
summary = summarizer(all_reviews, max_length=50, min_length=30, do_sample=False)

# Print the summary
print("Summary:")
print(summary[0]['summary_text'])

Summary:
 The ZEBRONICS Zeb-Jaguar Wireless Mouse is fantastic! It offers smooth and precise tracking, and the USB Nano Receiver makes it super convenient to use . The mouse is lightweight and ergonomic in your hands which
